In [23]:
import numpy as np
import cv2
from scipy.cluster.vq import kmeans,vq
from scipy.spatial import distance
import random

In [2]:
!pip install opencv-contrib-python==4.4.0.44

In [85]:
#Replace this code with proper image loading code from gdrive as following
#Load images as tuples (img, label)
#split into train test pair after shuffling
#trainX, trainY, testX, testY is obtained

imgs = []

for i in range(1,11):
  name = 'image_{:04d}.jpg'.format(i)
  im = cv2.imread(name,0)
  #im = im.astype('uint8')
  imgs.append((im, 0))

for i in range(89,103):
  im = cv2.imread('image_{:04d}.jpg'.format(i),0)
  #im = im.astype('uint8')
  imgs.append((im, 1))

random.shuffle(imgs)
train = list(zip(*imgs[:18]))
trainX = train[0]
trainY = train[1]

test = list(zip(*imgs[18:]))
testX = test[0]
testY = test[1]


In [81]:
def FeatureExtractor (imgs, extractor):
  imgsDescriptors = []
  allDescriptors = []
  for img in imgs:
    keypoint, descriptor = extractor.detectAndCompute(img,None)
    imgsDescriptors.append(descriptor)
    allDescriptors.extend(descriptor)

  return (allDescriptors, imgsDescriptors)

def CreateVocabulary (k, descriptors):  
  #convert descriptors to float?
  vocabulary, distortion = kmeans(descriptors, k, 1)
  return vocabulary

def ComputeHistogram (imgsDescriptors, vocabulary, k):
  histograms = []
  for imgDescriptors in imgsDescriptors:
    histogram = np.zeros(k)
    words, dist = vq(imgDescriptors, vocabulary)
    for w in words:
      histogram[w] += 1
    histograms.append(histogram)
  return histograms

def Predict (testHistograms, trainHistograms, trainY):
  testY = []
  for test in testHistograms:
    #use eucledian dist for now, change to chi square later
    dists = np.square(test[np.newaxis,:] - trainHistograms).sum(axis=1)
    mostLike = np.argmin(dists)
    testY.append(trainY[mostLike])

  return testY




In [95]:
sift = cv2.SIFT_create()
(allDescriptors, imgsDescriptorsTrain) = FeatureExtractor(trainX, sift)
k = 300
vocabulary = CreateVocabulary(k, allDescriptors)
histTrain = ComputeHistogram(imgsDescriptorsTrain, vocabulary, k)
(ignore, imgsDescriptorsTest) = FeatureExtractor(testX, sift)
histTest = ComputeHistogram(imgsDescriptorsTest, vocabulary, k)



In [96]:
testY

(1, 1, 0, 1, 0, 1)

In [97]:
Predict(histTest, histTrain, trainY)

[1, 1, 1, 0, 0, 1]

In [18]:
#Testing on single image

im = cv2.imread('image_0001.jpg',0)
sift = cv2.SIFT_create()
(allDescriptors, imgsDescriptors) = FeatureExtractor([im], sift)
k = 20
vocabulary = CreateVocabulary(k, allDescriptors)
histograms = ComputeHistogram(imgsDescriptors, vocabulary, k)



In [ ]:
# def FeatureExtractor (classToImgs, extractor):
#   #inputs images in the form {classLabel: [images]}
#   #outputs ( [all descriptors from all the images], {classLabel:[keypoints x 128 ndarray for each classLabel image]} )
#   classToImgDescriptors = {}
#   allDescriptors = []

#   for classLabel,images in classToImgs.items():
#     classToImgDescriptors[classLabel] = []
#     for img in images:
#       keypoint, descriptor = extractor.detectAndCompute(img,None)
#       classToImgDescriptors[classLabel].append(descriptor)
#       allDescriptors.extend(descriptor)

#   return (allDescriptors, classToImgDescriptors)



# def ComputeHistogram (classToImgDescriptors,vocabulary, k):
#   classToImageHistograms = {}
#   for classes, imgs in classToImgDescriptors.items():
#     classToImageHistograms[classes] = []
#     for img in imgs:
#       histogram = np.zeros(k)
#       words, dist = vq(img, vocabulary)
#       for w in words:
#         histogram[w] += 1
#       classToImageHistograms[classes].append(histogram)